# Getting started

In [1]:
# do basic imports and unpack McMurdo data

from pmagpy import ipmag
from pmagpy import pmag
from programs import new_builder as nb
from programs import data_model3
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
from programs.new_builder import Contribution

wdir = os.path.join(os.getcwd(), "3_0", "McMurdo")
infile = os.path.join(wdir, "lawrence09.v30.txt")
ipmag.download_magic(infile, overwrite=True)


/Users/nebula/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


working on:  'contribution'
1  records written to file  ./contribution.txt
contribution  data put in  ./contribution.txt
working on:  'locations'
5  records written to file  ./locations.txt
locations  data put in  ./locations.txt
working on:  'sites'
391  records written to file  ./sites.txt
sites  data put in  ./sites.txt
working on:  'samples'
1418  records written to file  ./samples.txt
samples  data put in  ./samples.txt
working on:  'specimens'
1374  records written to file  ./specimens.txt
specimens  data put in  ./specimens.txt
working on:  'measurements'
25470  records written to file  ./measurements.txt
measurements  data put in  ./measurements.txt
working on:  'ages'
99  records written to file  ./ages.txt
ages  data put in  ./ages.txt
working on:  'images'
431  records written to file  ./images.txt
images  data put in  ./images.txt
working on:  'criteria'
24  records written to file  ./criteria.txt
criteria  data put in  ./criteria.txt
location_1:  McMurdo
directory  ./Locat

True

# Several ways of creating a contribution

In [2]:
reload(nb)

# test out various ways of creating a contribution

#class Contribution(object):
#    """                                                                                                                   
#    A Contribution is a collection of MagicDataFrames,                                                                    
#    each of which corresponds to one MagIC table.                                                                         
#    The Contribution object also has methods for                                                                          
#    manipulating one or more tables in the contribution --                                                                
#    for example, renaming a site.                                                                                         
#    """
#    def __init__(self, directory, read_tables='all',
#                 custom_filenames=None, single_file=None):



# make contribution reading in all default filenames from working directory
con = nb.Contribution(wdir)
print 'tables created:', con.tables.keys()
print '-'

# make contribution with some custom filenames
con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt'})
print con
print 'tables created:', con.tables.keys()

# make contribution with custom filenames, and only read in the specimen table to start
con = Contribution(wdir, read_tables=['specimens'], custom_filenames={'sites': 'custom_sites.txt',
                                                                      'specimens': 'custom_specimens.txt'})
print 'tables created:', con.tables.keys()
print '-'

# make contribution with a single, mystery file (can be any datatype)
con = nb.Contribution(wdir, single_file='sites.txt')
print 'tables created:', con.tables.keys()
print '-'


-W- No such file: /Users/nebula/Python/PmagPy/3_0/McMurdo/specimens.txt
-W- No such file: /Users/nebula/Python/PmagPy/3_0/McMurdo/sites.txt
tables created: ['measurements', 'ages', 'locations', 'samples', 'criteria', 'images', 'contribution']
-
-W- No such file: /Users/nebula/Python/PmagPy/3_0/McMurdo/sites.txt
tables created: ['measurements', 'ages', 'locations', 'samples', 'criteria', 'images', 'contribution', 'specimens']
tables created: ['specimens']
-
-W- Bad file /Users/nebula/Python/PmagPy/3_0/McMurdo/sites.txt
tables created: []
-


In [3]:
# make McMurdo contribution, starting with specimens table

reload(nb)

con = nb.Contribution(wdir, read_tables=['specimens'], custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                                                         'sites': 'custom_sites.txt'})

print con.filenames
print con.tables.keys()

{'measurements': 'measurements.txt', 'ages': 'ages.txt', 'sites': 'custom_sites.txt', 'locations': 'locations.txt', 'samples': 'custom_samples.txt', 'criteria': 'criteria.txt', 'images': 'images.txt', 'contribution': 'contribution.txt', 'specimens': 'custom_specimens.txt'}
['specimens']


In [4]:
# then, add another table to the contribution
# here, we are providing data type but no filename
# this works because we already gave the custom sample filename when we created the contribution
# so the contribution already knows where to look (con.filenames)
con.add_magic_table('samples')
print con.tables.keys()

['specimens', 'samples']


In [5]:
# add another table to the same contribution
# this time, provide a filename but no data type

con.add_magic_table(dtype="unknown", fname="criteria.txt")
# criteria table now included
print con.tables.keys()

['specimens', 'samples', 'criteria']


# Functionality with a contribution

In [6]:
# create full McMurdo contribution

reload(nb)

con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})


## Rename an item

In [7]:
# rename one of the Contribution's sites
con.rename_item('sites', 'mc01', 'extra_special_site')
con.tables['sites'].df.ix[['extra_special_site']]
# all rows previously named 'mc01' are now named 'extra_special_site'

can only join an iterable


,age,age_sigma,age_unit,analyst_names,citations,criteria_names,description,dir_alpha95,dir_comp_name,dir_dec,...,vadm_n_samples,vadm_sigma,vdm,vdm_n_samples,vdm_sigma,vgp_dm,vgp_dp,vgp_lat,vgp_lon,vgp_n_samples
site_name,,,,,,,,,,,,,,,,,,,,,
extra_special_site,1.18,0.005,Ma,@ltauxe,This study,DE-SPEC,Direction included in Pmag_Results.,4.2,A,258.6,...,None,None,None,None,None,None,None,None,None,None
extra_special_site,1.18,0.005,Ma,@ltauxe,This study,IE-SPEC,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
extra_special_site,1.18,0.005,Ma,@klawrenc,This study,DE-SITE,"VGP: Site (geog. coord) A comp: mc01, Site VGP...",4.2,A,258.6,...,None,None,None,None,None,4.5,8.1,-67.3,95.2,7


In [8]:
# additionally, 'mc01' has been replaced in the location table under site_names
#con.tables['locations'].df.ix[["Osler Volcanics, Nipigon Strait, Lower Reversed"]][['site_names']]
con.tables['locations'].df[['site_names', 'site_names_list']]

,site_names,site_names_list
location_name,,
McMurdo,None,None
McMurdo,mc02 : mc03 : mc04 : mc06 : mc07 : mc08 : mc09...,"[mc02 , mc03 , mc04 , mc06 , mc07 , mc08 ..."
McMurdo,extra_special_site:mc102:mc103:mc105:mc109:mc1...,"[extra_special_site, mc102, mc103, mc105, mc10..."
McMurdo,extra_special_site:mc02:mc03:mc04:mc06:mc07:mc...,"[extra_special_site, mc02, mc03, mc04, mc06, m..."
McMurdo,mc08 : mc09 : mc105 : mc109 : mc111 : mc113 : ...,"[mc08 , mc09 , mc105 , mc109 , mc111 , mc..."


## Propagate data from one table into another

In [9]:
# normally, each table only has one relationship up (i.e., a measurement table will have specimen name, but not sample name)
# sometimes, you need to access location_name at the site level (for example)
# this function propagates names down through any available tables
# the code snippet below won't work if the Contribution can't access the sample and site files!

con.propagate_name_down('location_name', 'specimens')

# specimens table now has sample, site, and location_names
con.tables['specimens'].df[['specimen_name', 'sample_name', 'site_name', 'location_name']].head()

,specimen_name,sample_name,site_name,location_name
specimen_name,,,,
mc01a,mc01a,mc01a,extra_special_site,McMurdo
mc01a,mc01a,mc01a,extra_special_site,McMurdo
mc01b,mc01b,mc01b,extra_special_site,McMurdo
mc01b,mc01b,mc01b,extra_special_site,McMurdo
mc01c,mc01c,mc01c,extra_special_site,McMurdo


In [10]:
# this function propagates values from arbitrary columns down
# i.e., get sample-level azimuth into the measurements table
# note: this will NOT work with names (specimen_name, sample_name, etc.).  
# for those relationships, use the above function: propagate_name_down

reload(nb)
con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})

meas_container = con.tables['measurements']
meas_df = meas_container.df

meas_df = con.propagate_cols_down(['azimuth', 'dip', 'fake_col'], 'measurements', 'samples')
meas_df.head()[['azimuth', 'dip']]

-W- Column 'fake_col' isn't in samples table, skipping it


,azimuth,dip
measurement_name,,
mc01f-LP-DIR-AF1,94,-55
mc01f-LP-DIR-AF2,94,-55
mc01f-LP-DIR-AF3,94,-55
mc01f-LP-DIR-AF4,94,-55
mc01f-LP-DIR-AF5,94,-55


## Writing out a MagIC file

In [11]:
con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})
site_container = con.tables['sites']
site_container.write_magic_file(custom_name='fancy_sites.txt', dir_path='./3_0/McMurdo')


-I- overwriting /Users/nebula/Python/PmagPy/3_0/McMurdo/fancy_sites.txt


## Mucking around with measurements

In [12]:
con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})

meas_container = con.tables['measurements']
meas_data = meas_container.df

meas_data['treatment'] = meas_data['treat_ac_field'].where(cond=meas_data['treat_ac_field'] != "0", other=meas_data['treat_temp'])
meas_data[['treatment', 'treat_ac_field', 'treat_temp']]

meas_data['treat_ac_field'].ix[0] = None
meas_data['treat_ac_field'] = meas_data['treat_ac_field'].astype(float)

meas_data[['treatment', 'treat_ac_field', 'treat_temp']].head()


,treatment,treat_ac_field,treat_temp
measurement_name,,,
mc01f-LP-DIR-AF1,273,NaN,273
mc01f-LP-DIR-AF2,0.005,0.0050,273
mc01f-LP-DIR-AF3,0.0075,0.0075,273
mc01f-LP-DIR-AF4,0.01,0.0100,273
mc01f-LP-DIR-AF5,0.0125,0.0125,273


## Dealing with criteria

In [13]:
#  grab a copy of the criteria and sites table to play with
criteria = con.tables['criteria'].df.copy()
sites = con.tables['sites'].df.copy()
locations = con.tables['locations'].df.copy()
specimens = con.tables['specimens'].df.copy()
samples = con.tables['samples'].df.copy()
sites.head()

,age,age_sigma,age_unit,analyst_names,citations,criteria_names,description,dir_alpha95,dir_comp_name,dir_dec,...,vadm_n_samples,vadm_sigma,vdm,vdm_n_samples,vdm_sigma,vgp_dm,vgp_dp,vgp_lat,vgp_lon,vgp_n_samples
site_name,,,,,,,,,,,,,,,,,,,,,
mc01,1.18,0.005,Ma,@ltauxe,This study,DE-SPEC,Direction included in Pmag_Results.,4.2,A,258.6,...,None,None,None,None,None,None,None,None,None,None
mc01,1.18,0.005,Ma,@ltauxe,This study,IE-SPEC,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
mc01,1.18,0.005,Ma,@klawrenc,This study,DE-SITE,"VGP: Site (geog. coord) A comp: mc01, Site VGP...",4.2,A,258.6,...,None,None,None,None,None,4.5,8.1,-67.3,95.2,7
mc02,0.33,0.01,Ma,@ltauxe,This study,DE-SPEC,Direction included in Pmag_Results.,2.1,A,328.6,...,None,None,None,None,None,None,None,None,None,None
mc02,0.33,0.01,Ma,@klawrenc,This study,DE-SITE,"VGP: Site (geog. coord) A comp: mc02, Site VGP...",2.1,A,328.6,...,None,None,None,None,None,2.5,4.1,79,101.2,6


In [14]:
# get all criteria for sites

# only criteria with 'site' in table_column_name
cond = criteria.index.str.contains('site')
site_crit = criteria[cond].copy()
# remove table name from index
site_crit.index = site_crit.index.str.replace('sites.', '')
site_crit.index.name = 'column_name'


cols = site_crit.index
list(cols)
cols = list(cols)
cols.append('criteria_names')
site_crit

,citations,criterion_name,criterion_operation,criterion_value,description,table_column_name
column_name,,,,,,
dir_alpha95,This study,DE-SITE,<=,180,Criteria for selection of site direction,sites.dir_alpha95
dir_n_samples,This study,DE-SITE,>=,5,Criteria for selection of site direction,sites.dir_n_samples
dir_n_specimens_lines,This study,DE-SITE,>=,4,Criteria for selection of site direction,sites.dir_n_specimens_lines
dir_k,This study,DE-SITE,>=,50,Criteria for selection of site direction,sites.dir_k
int_abs_sigma,This study,IE-SITE,<=,-1,Criteria for selection of site intensity,sites.int_abs_sigma
int_abs_sigma_perc,This study,IE-SITE,<=,15,Criteria for selection of site intensity,sites.int_abs_sigma_perc
int_n_samples,This study,IE-SITE,>=,2,Criteria for selection of site intensity,sites.int_n_samples
dir_polarity,This study,NPOLE,=,n,Criteria for inclusion in normal mean,sites.dir_polarity
dir_polarity,This study,RPOLE,=,r,Criteria for inclusion in reverse mean,sites.dir_polarity


In [15]:
sites.head()[cols]

,dir_alpha95,dir_n_samples,dir_n_specimens_lines,dir_k,int_abs_sigma,int_abs_sigma_perc,int_n_samples,dir_polarity,dir_polarity,criteria_names
site_name,,,,,,,,,,
mc01,4.2,7,6,215.9,None,None,None,r,r,DE-SPEC
mc01,None,None,None,None,0.000003374,20.4,4,None,None,IE-SPEC
mc01,4.2,7,None,215.9,None,None,None,None,None,DE-SITE
mc02,2.1,6,5,1079.2,None,None,None,n,n,DE-SPEC
mc02,2.1,6,None,1079.2,None,None,None,None,None,DE-SITE


In [16]:
# create string --> operator conversion    
import operator
ops = {"<": operator.lt, ">": operator.gt, "==": operator.eq, "<=": operator.le, ">=": operator.gt}

# function for applying criteria
def apply_crit(series, crit_series, criteria_type):
    """
    Apply 1 criterion (i.e., 1 row of the criteria table) to another table.
    Return series with boolean values for whether the row passes.
    """
    col_name = crit_series.name
    #print col_name
    # if there's no value, pass == True??  or == False?
    if not series[col_name]:
        return "{} not in row".format(col_name)
    # if we're missing criteria names, then what??
    elif not series['criteria_names']:
        return "no value in criteria_names"
    elif criteria_type not in series['criteria_names']:
        return "{} not in criteria_names for this row".format(criteria_type)
    crit_name = crit_series['criterion_name']
    crit_value = float(crit_series['criterion_value'])
    op_str = crit_series['criterion_operation']
    op = ops[op_str]
    value = float(series[col_name])
    #print value, op_str, crit_value
    result = op(value, crit_value)
    #print op
    #print result
    return result





In [17]:
# apply a criterion to a table
crit_name = 'dir_alpha95'
pass_name = 'pass_' + crit_name
crit1 = site_crit.ix[crit_name]
sites[pass_name] = sites.apply(apply_crit, axis=1, args=(crit1, 'DE-SITE'))
cond = sites[pass_name] == True
sites[cond][[crit_name, pass_name]].head()

,dir_alpha95,pass_dir_alpha95
site_name,,
mc01,4.2,True
mc02,2.1,True
mc03,2.3,True
mc04,4.6,True
mc06,4.8,True


In [18]:
for crit_name in site_crit.index[1:]:
    crit_series = site_crit.ix[crit_name]
    # if there are multiple records for a single crit_name, ignore that one
    if not isinstance(crit_series, pd.Series):
        continue
    sites['pass_' + crit_name] = sites.apply(apply_crit, axis=1, args=(crit_series, "DE-SITE"))

    
cond = sites.columns.str.contains('pass')
sites.head()
sites[sites.columns[cond]].head()

,pass_dir_alpha95,pass_dir_n_samples,pass_dir_n_specimens_lines,pass_dir_k,pass_int_abs_sigma,pass_int_abs_sigma_perc,pass_int_n_samples
site_name,,,,,,,
mc01,DE-SITE not in criteria_names for this row,DE-SITE not in criteria_names for this row,DE-SITE not in criteria_names for this row,DE-SITE not in criteria_names for this row,int_abs_sigma not in row,int_abs_sigma_perc not in row,int_n_samples not in row
mc01,dir_alpha95 not in row,dir_n_samples not in row,dir_n_specimens_lines not in row,dir_k not in row,DE-SITE not in criteria_names for this row,DE-SITE not in criteria_names for this row,DE-SITE not in criteria_names for this row
mc01,True,True,dir_n_specimens_lines not in row,True,int_abs_sigma not in row,int_abs_sigma_perc not in row,int_n_samples not in row
mc02,DE-SITE not in criteria_names for this row,DE-SITE not in criteria_names for this row,DE-SITE not in criteria_names for this row,DE-SITE not in criteria_names for this row,int_abs_sigma not in row,int_abs_sigma_perc not in row,int_n_samples not in row
mc02,True,True,dir_n_specimens_lines not in row,True,int_abs_sigma not in row,int_abs_sigma_perc not in row,int_n_samples not in row


In [19]:
# create full McMurdo contribution

reload(nb)

con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})


criteria = con.tables['criteria'].df.copy()


#criteria
#print 'criteria_names' in specimens.columns
#print 'criteria_names' in samples.columns
#print 'criteria_names' in sites.columns
#print 'criteria_names' in locations.columns
#sites[['criteria_names']]


def apply_crit(series, crit_series):#, criteria_type):
    """
    Apply 1 criterion (i.e., 1 row of the criteria table) to another table.
    Return series with boolean values for whether the row passes.
    """
    col_name = crit_series.name
    # if there's no value, pass == True
    if not series[col_name]:
        return True
        #return "{} not in row".format(col_name)
    # if there is a value, test that it is within correct limits
    crit_name = crit_series['criterion_name']
    crit_value = float(crit_series['criterion_value'])
    op_str = crit_series['criterion_operation']
    op = ops[op_str]
    value = float(series[col_name])
    result = op(value, crit_value)
    return result



def add_criteria_named(category_name, dtype):
    df = con.tables[dtype].df
    criteria_subset = criteria[criteria['criterion_name'] == category_name]
    criteria_subset.index = criteria_subset.index.str.replace(dtype + '.', '')
    pass_col_names = []
    for crit_name, crit_row in criteria_subset.iterrows():
        #print 'crit_name', crit_name
        col_name = category_name + "_" + crit_name + "_pass"
        #print 'col_name', col_name
        pass_col_names.append(col_name)
        df[col_name] = df.apply(apply_crit, args=(crit_row,), axis=1)
    return pass_col_names
    

#DE_SPEC = criteria[criteria['criterion_name'] == 'DE-SPEC']
#DE_SPEC.index = DE_SPEC.index.str.replace('specimens.', '')
#pass_col_names = []
#for crit_name, crit_row in DE_SPEC.iterrows():
#    #print 'crit_name', crit_name
#    col_name = 'DE-SPEC_' + crit_name + "_pass"
#    #print 'col_name', col_name
#    pass_col_names.append(col_name)
#    specimens[col_name] = specimens.apply(apply_crit, args=(crit_row,), axis=1)
    
    
#specimens_container = con.tables['specimens']
dtype = 'specimens'
criteria_name = 'IE-SPEC'
pass_col = criteria_name + "_pass"
pass_col_names = add_criteria_named(criteria_name, dtype)
df = con.tables[dtype].df

#df.ix[:, 13:]
col_names = df.columns[df.columns.str.contains(criteria_name)]
df[df[col_names].all(1)].head()#[col_names]

#cond1 = df[pass_col_names[0]]
#cond2 = df[pass_col_names[1]]
#cond3 = df[pass_col_names[2]]

#cond = cond1 & cond2 & cond3
#df[pass_col] = cond

## all specimens that pass all DE-SPEC criteria
#df[df[pass_col]].index



,analyst_names,aniso_ftest,aniso_ftest12,aniso_ftest23,aniso_s,aniso_s_mean,aniso_s_n_measurements,aniso_s_sigma,aniso_s_unit,aniso_tilt_correction,...,software_packages,specimen_name,IE-SPEC_int_mad_free_pass,IE-SPEC_int_q_pass,IE-SPEC_int_fvds_pass,IE-SPEC_int_b_beta_pass,IE-SPEC_int_dang_pass,IE-SPEC_int_md_pass,IE-SPEC_int_drats_pass,IE-SPEC_int_n_ptrm_pass
specimen_name,,,,,,,,,,,,,,,,,,,,,
mc01a,@ltauxe,None,None,None,None,None,None,None,None,None,...,pmagpy-2.60,mc01a,True,True,True,True,True,True,True,True
mc01a,@ltauxe,None,None,None,None,None,None,None,None,None,...,pmagpy-2.58,mc01a,True,True,True,True,True,True,True,True
mc01b,@ltauxe,None,None,None,None,None,None,None,None,None,...,pmagpy-2.58,mc01b,True,True,True,True,True,True,True,True
mc01b,@ltauxe,None,None,None,None,None,None,None,None,None,...,pmagpy-2.60,mc01b,True,True,True,True,True,True,True,True
mc01c,@ltauxe,None,None,None,None,None,None,None,None,None,...,pmagpy-2.58,mc01c,True,True,True,True,True,True,True,True


# Cleanup

In [20]:
!rm sites.txt samples.txt specimens.txt measurements.txt ages.txt contribution.txt images.txt criteria.txt locations.txt
!rm *.png
!rm *.jpg

rm: *.png: No such file or directory
rm: *.jpg: No such file or directory
